In [15]:
from syft import Worker, MessageStatus
from syft.core.node.new.project import Project
import syft as sy

In [16]:
worker = Worker.named("test-domain-1", processes=1, reset=False)

SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Starting Worker: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.Data

In [17]:
domain_client = worker.guest_client.login(email="info@openmined.org", password="changethis")

# Get messages

In [18]:
messages = domain_client.api.services.messages.get_all_for_status(MessageStatus.UNDELIVERED)

In [19]:
messages

,type,id,subject,status,created_at,linked_obj
0,syft.core.node.new.messages.Message,8d9f4facccbc4732a3a3518f0b2967db,Project Approval,MessageStatus.UNDELIVERED,2023-04-17 09:50:50,<<class 'syft.core.node.new.project.Project'>:...
1,syft.core.node.new.messages.Message,6aad66e5ba694fc992f8bde0611e3bce,Approval Request,MessageStatus.UNDELIVERED,2023-04-17 09:50:49,<<class 'syft.core.node.new.request.Request'>:...


In [20]:
project_message = [x for x in messages if issubclass(x.linked_obj.object_type, Project)][0]

In [21]:
request = project_message.link.requests[0]
func = request.changes[0].link
op = func.output_policy_type

In [22]:
func

```python
class UserCode:
  id: str = 3d953d3609c1433b9229d62d8920f9d2
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  user_verify_key: str = 00473a5412de651c4198edae3465fcb9c945caa123dfc4070da5e894ad52e77f
  raw_code: str = "@sy.syft_function(input_policy=sy.ExactMatch(complaints=asset.mock),
                  output_policy=sy.SingleExecutionExactOutput())
def get_counts(complaints):
    is_noise = complaints['Complaint Type'] == "Noise - Street/Sidewalk"
    noise_complaints = complaints[is_noise]
    noise_complaint_counts = noise_complaints['Borough'].value_counts()
    complaint_counts = complaints['Borough'].value_counts()
    return noise_complaint_counts / complaint_counts.astype(float)
"
  input_policy_type: str = <class 'syft.core.node.new.policy.ExactMatch'>
  input_policy_init_kwargs: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): {'complaints': <UID: 8ce7b2c4e2564b9aa8c113e046e4352d>}}
  input_policy_state: str = b''
  output_policy_type: str = <class 'syft.core.node.new.policy.OutputPolicyExecuteOnce'>
  output_policy_init_kwargs: str = {}
  output_policy_state: str = b''
  parsed_code: str = "

def user_func_get_counts_00473a5412de651c4198edae3465fcb9c945caa123dfc4070da5e894ad52e77f_701f23755a69b9f3f6d34d6a02c53686edbd4a524ddd4cb2d4815c33b33a5b20(complaints):

    def get_counts(complaints):
        is_noise = (complaints['Complaint Type'] == 'Noise - Street/Sidewalk')
        noise_complaints = complaints[is_noise]
        noise_complaint_counts = noise_complaints['Borough'].value_counts()
        complaint_counts = complaints['Borough'].value_counts()
        return (noise_complaint_counts / complaint_counts.astype(float))
    result = get_counts(complaints=complaints)
    return result
"
  service_func_name: str = "get_counts"
  unique_func_name: str = "user_func_get_counts_00473a5412de651c4198edae3465fcb9c945caa123dfc4070da5e894ad52e77f_701f23755a69b9f3f6d34d6a02c53686edbd4a524ddd4cb2d4815c33b33a5b20"
  user_unique_func_name: str = "user_func_get_counts_00473a5412de651c4198edae3465fcb9c945caa123dfc4070da5e894ad52e77f"
  code_hash: str = "701f23755a69b9f3f6d34d6a02c53686edbd4a524ddd4cb2d4815c33b33a5b20"
  signature: str = (complaints)
  status: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.SUBMITTED: 'submitted'>}
  input_kwargs: str = ['complaints']
  enclave_metadata: str = None

```

In [23]:
get_counts_user_func = func.unsafe_function

In [24]:
real_data = domain_client.datasets[0].assets[0].data

In [25]:
real_result = get_counts_user_func(complaints=real_data)

In [26]:
real_result[:3]

BRONX        0.014833
BROOKLYN     0.013864
MANHATTAN    0.037755
Name: Borough, dtype: float64

In [27]:
result = request.accept_by_depositing_result(real_result)
result
assert isinstance(result, sy.SyftSuccess)